<a href="https://colab.research.google.com/github/AI4Finance-LLC/FinRL-Library/blob/master/FinRL_ensemble_stock_trading_ICAIF_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading Using Ensemble Strategy

Tutorials to use OpenAI DRL to trade multiple stocks using ensemble strategy in one Jupyter Notebook | Presented at ICAIF 2020

* This notebook is the reimplementation of our paper: Deep Reinforcement Learning for Automated Stock Trading: An Ensemble Strategy, using FinRL.
* Check out medium blog for detailed explanations: https://medium.com/@ai4finance/deep-reinforcement-learning-for-automated-stock-trading-f1dad0126a02
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [2]:
# ## install finrl library
# !pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config import config
from finrl.marketdata.yahoodownloader import YahooDownloader
from finrl.preprocessing.preprocessors import FeatureEngineer
from finrl.preprocessing.data import data_split
from finrl.env.env_stocktrading import StockTradingEnv
from finrl.model.models import DRLAgent,DRLEnsembleAgent
from finrl.trade.backtest import backtest_stats, backtest_plot, get_daily_return, get_baseline
from stable_baselines3 import A2C

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

<a id='1.4'></a>
## 2.4. Create Folders

In [4]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [5]:
df = pd.read_csv("C:/Users/Mcian/Downloads/day_trader_bot/dukascopy/BTCUSD_Candlestick_1_M_BID_07.05.2017-16.03.2018.csv")
df = pd.concat([df, 
                pd.read_csv("C:/Users/Mcian/Downloads/day_trader_bot/dukascopy/BTCUSD_Candlestick_1_M_BID_17.03.2018-17.03.2021.csv")])

df['tic'] = 'BTCUSD'

In [6]:
if False:
    df = df.iloc[-100000:]

In [9]:
df['Local time'] = pd.to_datetime(df['Local time'])
df['date'] = df['Local time'].apply(lambda x: x.isoformat())
df['day'] = df['Local time'].apply(lambda x: x.weekday())
df['hour'] = df['Local time'].apply(lambda x: x.hour+x.minute/60)
# del df['Local time']
df.columns = [c.lower() for c in df.columns]
df.reset_index(drop=True, inplace=True)

In [10]:
df.head()

,local time,open,high,low,close,volume,tic,date,day,hour
0,2017-07-05 00:00:00+07:00,1555,1555,1555,1555,0.0,BTCUSD,2017-07-05T00:00:00+07:00,2,0.000000
1,2017-07-05 00:01:00+07:00,1555,1555,1555,1555,0.0,BTCUSD,2017-07-05T00:01:00+07:00,2,0.016667
2,2017-07-05 00:02:00+07:00,1555,1555,1555,1555,0.0,BTCUSD,2017-07-05T00:02:00+07:00,2,0.033333
3,2017-07-05 00:03:00+07:00,1555,1555,1555,1555,0.0,BTCUSD,2017-07-05T00:03:00+07:00,2,0.050000
4,2017-07-05 00:04:00+07:00,1555,1555,1555,1555,0.0,BTCUSD,2017-07-05T00:04:00+07:00,2,0.066667


In [11]:
df.tail()

,local time,open,high,low,close,volume,tic,date,day,hour
2031595,2021-03-17 19:55:00+07:00,59031,59076,59027,59075,0.0,BTCUSD,2021-03-17T19:55:00+07:00,2,19.916667
2031596,2021-03-17 19:56:00+07:00,59074,59085,59056,59067,0.0,BTCUSD,2021-03-17T19:56:00+07:00,2,19.933333
2031597,2021-03-17 19:57:00+07:00,59067,59088,59044,59057,0.0,BTCUSD,2021-03-17T19:57:00+07:00,2,19.950000
2031598,2021-03-17 19:58:00+07:00,59058,59063,59031,59043,0.0,BTCUSD,2021-03-17T19:58:00+07:00,2,19.966667
2031599,2021-03-17 19:59:00+07:00,59043,59043,58963,58963,0.0,BTCUSD,2021-03-17T19:59:00+07:00,2,19.983333


In [12]:
df.shape

(2031600, 10)

In [13]:
df.sort_values(['date','tic']).head()

,local time,open,high,low,close,volume,tic,date,day,hour
36000,2017-01-06 00:00:00+07:00,2384,2399,2384,2390,1.2497,BTCUSD,2017-01-06T00:00:00+07:00,4,0.000000
36001,2017-01-06 00:01:00+07:00,2390,2392,2389,2392,0.9095,BTCUSD,2017-01-06T00:01:00+07:00,4,0.016667
36002,2017-01-06 00:02:00+07:00,2393,2401,2392,2392,1.1128,BTCUSD,2017-01-06T00:02:00+07:00,4,0.033333
36003,2017-01-06 00:03:00+07:00,2392,2401,2392,2392,0.9967,BTCUSD,2017-01-06T00:03:00+07:00,4,0.050000
36004,2017-01-06 00:04:00+07:00,2393,2400,2390,2392,1.4063,BTCUSD,2017-01-06T00:04:00+07:00,4,0.066667


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [ ]:
config.TECHNICAL_INDICATORS_LIST

In [15]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_turbulence=False,
                    user_defined_feature=False)

processed = fe.preprocess_data(df)

Successfully added technical indicators


In [16]:
if False:
    list_ticker = processed["tic"].unique().tolist()
    list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
    combination = list(itertools.product(list_date,list_ticker))

    processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
    processed_full = processed_full[processed_full['date'].isin(processed['date'])]
    processed_full = processed_full.sort_values(['date','tic'])

    processed_full = processed_full.fillna(0)
else:
    processed_full = processed

In [17]:
processed_full.sample(5)

,local time,open,high,low,close,volume,tic,date,day,hour,...,boll_lb,rsi_10,cci_30,dx_30,close_50_sma,close_200_sma,atr,close_9_ema,close_20_ema,kdjk
795544,2018-11-10 10:04:00+07:00,6211,6211,6202,6202,0.0003,BTCUSD,2018-11-10T10:04:00+07:00,5,10.066667,...,6194.238310,48.582081,62.325581,13.943826,6201.30,6200.620,8.616183e+00,6203.482962,6202.770836,5.580474e+01
976273,2019-03-15 21:13:00+07:00,3838,3838,3838,3838,0.0000,BTCUSD,2019-03-15T21:13:00+07:00,4,21.216667,...,3837.996706,29.605540,1000.000000,72.076342,3838.00,3838.000,6.743354e-15,3838.000000,3838.000000,6.085305e-74
130357,2017-07-16 11:37:00+07:00,2158,2158,2158,2158,0.0000,BTCUSD,2017-07-16T11:37:00+07:00,6,11.616667,...,2157.998069,37.219614,-1000.000000,15.517029,2158.00,2158.000,3.667393e-88,2158.000000,2158.000000,4.940656e-324
627863,2018-07-16 22:23:00+07:00,6650,6650,6649,6649,1.3075,BTCUSD,2018-07-16T22:23:00+07:00,0,22.383333,...,6634.442053,59.053131,49.864824,2.418870,6650.28,6667.380,3.399143e+00,6646.584583,6645.740554,8.179715e+01
1772999,2020-09-19 04:59:00+07:00,11122,11123,11121,11123,0.0010,BTCUSD,2020-09-19T04:59:00+07:00,5,4.983333,...,10974.649066,85.357224,176.832141,65.961331,11013.34,10950.115,1.110968e+01,11087.953817,11061.131054,8.870213e+01


In [107]:
if True:
    processed_full.to_csv('processed_full_BTC.csv')

In [5]:
if True:    
    processed_full = pd.read_csv('processed_full_BTC.csv')
    del processed_full['Unnamed: 0']

In [6]:
processed_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031600 entries, 0 to 2031599
Data columns (total 22 columns):
 #   Column         Dtype  
---  ------         -----  
 0   local time     object 
 1   open           int64  
 2   high           int64  
 3   low            int64  
 4   close          int64  
 5   volume         float64
 6   tic            object 
 7   date           object 
 8   day            int64  
 9   hour           float64
 10  macd           float64
 11  boll_ub        float64
 12  boll_lb        float64
 13  rsi_10         float64
 14  cci_30         float64
 15  dx_30          float64
 16  close_50_sma   float64
 17  close_200_sma  float64
 18  atr            float64
 19  close_9_ema    float64
 20  close_20_ema   float64
 21  kdjk           float64
dtypes: float64(14), int64(5), object(3)
memory usage: 341.0+ MB


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

In [7]:
stock_dimension = len(processed_full.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension + 1
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 1, State Space: 16


In [8]:
def cost_function_BTC(volume):
    if volume < 10000:
        return 0.005
    elif volume < 50000:
        return 0.0035
    elif volume < 100000:
        return 0.0025
    elif volume < 1000000:
        return 0.002
    elif volume < 10000000:
        return 0.0018
    elif volume < 50000000:
        return 0.0015
    elif volume < 100000000:
        return 0.001
    elif volume < 300000000:
        return 0.0007
    elif volume < 500000000:
        return 0.0005
    elif volume < 1000000000:
        return 0.0004
    elif volume > 1000000000:
        return 0.0004

In [9]:
env_kwargs = {
    "hmax": 0.95, 
    "initial_amount": 50000, 
    "buy_cost_pct": cost_function_BTC, 
    "sell_cost_pct": cost_function_BTC, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4,
    "print_verbosity":5
    
}

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

* In this notebook, we are training and validating 3 agents (A2C, PPO, DDPG) using Rolling-window Ensemble Method ([reference code](https://github.com/AI4Finance-LLC/Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020/blob/80415db8fa7b2179df6bd7e81ce4fe8dbf913806/model/models.py#L92))

In [10]:
val_size = int(len(processed_full)*0.1)

train_start = processed_full.iloc[:-val_size]['date'].min()
train_end = processed_full.iloc[:-val_size]['date'].max()
val_test_start = processed_full.iloc[-val_size:]['date'].min()
val_test_end = processed_full.iloc[-val_size:]['date'].max()

In [11]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.01,
                    'learning_rate': 0.0005
                    }

timesteps_dict = {'a2c' : 100_000, 
                 'ppo' : 100_000, 
                 'ddpg' : 50_000
                 }

In [16]:
temp = processed_full.iloc[:1000].copy()
temp['local time'] = pd.to_datetime(temp['local time'])

In [17]:
env = StockTradingEnv(temp, **env_kwargs)

In [18]:
agent = DRLAgent(env)

In [19]:
model = agent.get_model(
    'a2c',
    policy="MlpPolicy",
    verbose=1,
    model_kwargs=A2C_model_kwargs,
)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0005}
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
agent.train_model(
    model,
    tb_log_name=f"{config.TENSORBOARD_LOG_DIR}/a2c",
)

Logging to tensorboard_log/a2c\tensorboard_log/a2c_1
------------------------------------
| time/                 |          |
|    fps                | 5        |
|    iterations         | 100      |
|    time_elapsed       | 89       |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.46    |
|    explained_variance | 5.96e-08 |
|    learning_rate      | 0.0005   |
|    n_updates          | 99       |
|    policy_loss        | 0.00236  |
|    std                | 1.05     |
|    value_loss         | 2.85e-06 |
------------------------------------
-------------------------------------
| environment/          |           |
|    portfolio_value    | 4.39e+04  |
|    total_cost         | 6.08e+03  |
|    total_reward       | -6.07e+03 |
|    total_reward_pct   | -12.1     |
|    total_trades       | 938       |
| rollout/              |           |
|    ep_len_mean        | 940       |
|    ep_rew_mean        | -0.612    |
| time/     

In [ ]:
trade_env = DummyVecEnv([lambda: StockTradingEnv(trade_data,
                                                 self.stock_dim,
                                                 self.hmax,
                                                 self.initial_amount,
                                                 self.buy_cost_pct,
                                                 self.sell_cost_pct,
                                                 self.reward_scaling,
                                                 self.state_space,
                                                 self.action_space,
                                                 self.tech_indicator_list,
                                                 turbulence_threshold=turbulence_threshold,
                                                 initial=initial,
                                                 previous_state=last_state,
                                                 model_name=name,
                                                 mode='trade',
                                                 iteration=iter_num,
                                                 print_verbosity=self.print_verbosity)])

In [22]:
agent.DRL_prediction(model, env)

hit end!


IndexError: list index out of range

In [ ]:
df_summary = ensemble_agent.run_ensemble_strategy(A2C_model_kwargs,
                                                 PPO_model_kwargs,
                                                 DDPG_model_kwargs,
                                                 timesteps_dict)

============Start Ensemble Strategy============
======Model training from:  2017-01-06T00:00:00+07:00 to  2020-10-27T17:01:00+07:00
======A2C Training========
{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0005}
Using cuda device
Logging to tensorboard_log/a2c\a2c_126_11
------------------------------------
| time/                 |          |
|    fps                | 6        |
|    iterations         | 100      |
|    time_elapsed       | 74       |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.45    |
|    explained_variance | -88.7    |
|    learning_rate      | 0.0005   |
|    n_updates          | 99       |
|    policy_loss        | -0.193   |
|    std                | 1.03     |
|    value_loss         | 0.0128   |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 6        |
|    iterations         | 200      |
|    time_elapsed   

-------------------------------------
| time/                 |           |
|    fps                | 6         |
|    iterations         | 1600      |
|    time_elapsed       | 1192      |
|    total_timesteps    | 8000      |
| train/                |           |
|    entropy_loss       | -1.76     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0005    |
|    n_updates          | 1599      |
|    policy_loss        | 0.00746   |
|    std                | 1.41      |
|    value_loss         | 2.25e-05  |
-------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 6        |
|    iterations         | 1700     |
|    time_elapsed       | 1267     |
|    total_timesteps    | 8500     |
| train/                |          |
|    entropy_loss       | -1.78    |
|    explained_variance | -1.53    |
|    learning_rate      | 0.0005   |
|    n_updates          | 1699     |
|    policy_loss       

------------------------------------
| time/                 |          |
|    fps                | 6        |
|    iterations         | 3100     |
|    time_elapsed       | 2307     |
|    total_timesteps    | 15500    |
| train/                |          |
|    entropy_loss       | -2.24    |
|    explained_variance | -5.15    |
|    learning_rate      | 0.0005   |
|    n_updates          | 3099     |
|    policy_loss        | -0.0151  |
|    std                | 2.26     |
|    value_loss         | 1.8e-05  |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 6         |
|    iterations         | 3200      |
|    time_elapsed       | 2382      |
|    total_timesteps    | 16000     |
| train/                |           |
|    entropy_loss       | -2.28     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0005    |
|    n_updates          | 3199      |
|    policy_loss        | 0

------------------------------------
| time/                 |          |
|    fps                | 6        |
|    iterations         | 4600     |
|    time_elapsed       | 3420     |
|    total_timesteps    | 23000    |
| train/                |          |
|    entropy_loss       | -2.86    |
|    explained_variance | 5.96e-08 |
|    learning_rate      | 0.0005   |
|    n_updates          | 4599     |
|    policy_loss        | 0.000351 |
|    std                | 4.25     |
|    value_loss         | 3.15e-08 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 6        |
|    iterations         | 4700     |
|    time_elapsed       | 3494     |
|    total_timesteps    | 23500    |
| train/                |          |
|    entropy_loss       | -2.91    |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 4699     |
|    policy_loss        | 0.00351  |
|

-------------------------------------
| time/                 |           |
|    fps                | 6         |
|    iterations         | 6100      |
|    time_elapsed       | 4535      |
|    total_timesteps    | 30500     |
| train/                |           |
|    entropy_loss       | -3.56     |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0005    |
|    n_updates          | 6099      |
|    policy_loss        | -0.00022  |
|    std                | 8.49      |
|    value_loss         | 4.55e-09  |
-------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 6        |
|    iterations         | 6200     |
|    time_elapsed       | 4609     |
|    total_timesteps    | 31000    |
| train/                |          |
|    entropy_loss       | -3.6     |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 6199     |
|    policy_loss       

...

------------------------------------
| time/                 |          |
|    fps                | 6        |
|    iterations         | 19600    |
|    time_elapsed       | 14572    |
|    total_timesteps    | 98000    |
| train/                |          |
|    entropy_loss       | -10.1    |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 19599    |
|    policy_loss        | 7.31e-06 |
|    std                | 6.04e+03 |
|    value_loss         | 6.32e-13 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 6        |
|    iterations         | 19700    |
|    time_elapsed       | 14646    |
|    total_timesteps    | 98500    |
| train/                |          |
|    entropy_loss       | -10.2    |
|    explained_variance | 0        |
|    learning_rate      | 0.0005   |
|    n_updates          | 19699    |
|    policy_loss        | 0.00956  |
|

------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 9            |
|    time_elapsed         | 2721         |
|    total_timesteps      | 18432        |
| train/                  |              |
|    approx_kl            | 0.0013498039 |
|    clip_fraction        | 0.0082       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.44        |
|    explained_variance   | 0.0883       |
|    learning_rate        | 0.00025      |
|    loss                 | -0.00673     |
|    n_updates            | 80           |
|    policy_gradient_loss | -0.00132     |
|    std                  | 1.03         |
|    value_loss           | 0.000255     |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 10           |
|    time_e

------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 20           |
|    time_elapsed         | 6044         |
|    total_timesteps      | 40960        |
| train/                  |              |
|    approx_kl            | 0.0023599383 |
|    clip_fraction        | 0.0128       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.5         |
|    explained_variance   | -5.84        |
|    learning_rate        | 0.00025      |
|    loss                 | -0.0196      |
|    n_updates            | 190          |
|    policy_gradient_loss | -0.00148     |
|    std                  | 1.08         |
|    value_loss           | 3.81e-05     |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 21           |
|    time_e

------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 31           |
|    time_elapsed         | 9361         |
|    total_timesteps      | 63488        |
| train/                  |              |
|    approx_kl            | 0.0003716307 |
|    clip_fraction        | 0.0329       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.49        |
|    explained_variance   | 0.096        |
|    learning_rate        | 0.00025      |
|    loss                 | -0.0317      |
|    n_updates            | 300          |
|    policy_gradient_loss | -0.00124     |
|    std                  | 1.07         |
|    value_loss           | 1.19e-06     |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 32           |
|    time_e

-----------------------------------------
| time/                   |             |
|    fps                  | 6           |
|    iterations           | 42          |
|    time_elapsed         | 12680       |
|    total_timesteps      | 86016       |
| train/                  |             |
|    approx_kl            | 0.003081485 |
|    clip_fraction        | 0.0824      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.52       |
|    explained_variance   | -0.251      |
|    learning_rate        | 0.00025     |
|    loss                 | 0.00043     |
|    n_updates            | 410         |
|    policy_gradient_loss | -0.00255    |
|    std                  | 1.1         |
|    value_loss           | 1.96e-08    |
-----------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 43           |
|    time_elapsed         | 12

In [ ]:
df_summary

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [ ]:
unique_trade_date = processed_full[(processed_full.date > val_test_start)&(processed_full.date <= val_test_end)].date.unique()

In [ ]:
df_trade_date = pd.DataFrame({'datadate':unique_trade_date})

df_account_value=pd.DataFrame()
for i in range(rebalance_window+validation_window, len(unique_trade_date)+1,rebalance_window):
    temp = pd.read_csv('results/account_value_trade_{}_{}.csv'.format('ensemble',i))
    df_account_value = df_account_value.append(temp,ignore_index=True)
sharpe=(252**0.5)*df_account_value.account_value.pct_change(1).mean()/df_account_value.account_value.pct_change(1).std()
print('Sharpe Ratio: ',sharpe)
df_account_value=df_account_value.join(df_trade_date[validation_window:].reset_index(drop=True))

In [ ]:
df_account_value.head()

In [ ]:
%matplotlib inline
df_account_value.account_value.plot()

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to IHSG===========")
%matplotlib inline
backtest_plot(df_account_value, 
             baseline_ticker = '^JKSE', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])

<a id='6.3'></a>
## 7.3 Baseline Stats

In [ ]:
print("==============Get Baseline Stats===========")
baseline_perf_stats=get_baseline('^JKSE',
                                  start = df_account_value.loc[0,'date'],
                                  end = df_account_value.loc[len(df_account_value)-1,'date'])

#### 